In [2]:
from pyspark.sql import SparkSession,Row,DataFrame
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import Window



# Create SparkSession
spark=SparkSession.builder.master("local[1]")\
        .appName("Music")\
        .config("spark.driver.extraClassPath","/Users/eduardoalberto/opt/spark/jars/mysql-connector-j-8.2.0.jar" ) \
        .getOrCreate()
sc = spark.sparkContext
spark.sparkContext.setLogLevel("OFF") 
print('PySpark Version :'+spark.version)
print('PySpark Version :'+spark.sparkContext.version)

spark

24/05/05 22:02:57 WARN Utils: Your hostname, Eduardos-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.3.5 instead (on interface en0)
24/05/05 22:02:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/05 22:02:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


PySpark Version :3.5.0
PySpark Version :3.5.0


In [3]:
tb_genres = spark.read.jdbc("jdbc:mysql://localhost:3306/myDbUser", "myDbUser.tb_genres",properties={"user": "root", "password": "mysql"})
tb_genres.show()


+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-----+--------------+--------------------+--------------------+--------------------+-----------+--------------+---------+--------------------+----------+
|danceability|energy|key|loudness|mode|speechiness|acousticness|instrumentalness|liveness|valence|tempo|          type|                 uri|          track_href|        analysis_url|duration_ms|time_signature|    genre|           song_name|     rowid|
+------------+------+---+--------+----+-----------+------------+----------------+--------+-------+-----+--------------+--------------------+--------------------+--------------------+-----------+--------------+---------+--------------------+----------+
|        0.51|  0.86|  1|   -5.10|   1|       0.07|        0.00|            0.00|    0.40|   0.38| NULL|audio_features|spotify:track:11T...|https://api.spoti...|https://api.spoti...|     310160|             4|   trance|                NULL|8589

### MUSIC

In [ ]:

df_music = spark.read.option("header", "true").option('inferSchema', 'true').csv('/Users/eduardoalberto/LoadFile/archive/data.csv')
df_music = df_music.selectExpr("cast(acousticness as decimal(3,2)) as acousticness",
                               "regexp_replace(artists,'[^a-zA-Z0-9]',' ') as artists",
                               "cast(danceability as decimal(3,2)) as danceability ",
                               "cast(energy as decimal(3,2)) as energy",
                               "duration_ms",
                               "explicit",
                               "id",
                               "cast(instrumentalness as decimal(3,2) ) as instrumentalness",
                               "key",
                               "cast (liveness as decimal(3,2) ) as liveness",
                               "cast(loudness as decimal(3,2)) as loudness",
                               "mode",
                               "name",
                               "popularity",
                               """ case
                                        when release_date in ('1928','2013') then '1928-01-01' 
                                        else release_date
                                    end as release_date
                               """,
                               "cast(speechiness as decimal(5,2)) as speechiness",
                               "cast(tempo as decimal(3,2)) as tempo",
                               "cast(valence as decimal(3,2)) as valence",
                               "year",
                               "current_date() as dat_ref_carga"
                               )\
                   .withColumn('release_date',expr("case when release_date = 'Whipped into Shape' then '1990-01-10' else release_date end"))\
                   .withColumn('danceability', expr(" case when trim(regexp_replace(danceability,'[^a-zA-Z0-9]',' ')) = 'Michael Rupert' then 0 else danceability end  "))\
                   .withColumn("rowid", monotonically_increasing_id()) 

#df_music.filter("release_date = 'Whipped into Shape'").show(300,False)
df_music.show()

### tracks

In [ ]:
tracks = spark.read.option("header", "true").option('inferSchema', 'true').csv('/Users/eduardoalberto/LoadFile/archive/tracks.csv')

df_tracks = tracks.selectExpr("id"
                             ,"name"
                             ,"popularity"
                             ,"duration_ms"
                             ,"explicit"
                             ,"regexp_replace('artists','[^a-zA-Z0-9]',' ') as artists "
                             ,"regexp_replace('id_artists','[^a-zA-Z0-9]',' ') as id_artists"
                             ,"case when length(release_date) = 4  then '1922-01-01' else release_date end as release_date "
                             ,"danceability"
                             ,"energy"
                             ,"key"
                             ,"loudness"
                             ,"mode"
                             ,"loudness"
                             ,"mode"
                             ,"speechiness"
                             ,"acousticness"
                             ,"instrumentalness"
                             ,"liveness"
                             ,"valence"
                             ,"tempo"
                             ,"time_signature")
df_tracks.show()

### GENRES

In [ ]:
genres = spark.read.option("header", "true").option('inferSchema', 'true').csv('/Users/eduardoalberto/LoadFile/archive/genres_v2.csv')
genres.selectExpr("cast(acousticness as decimal(3,2)) as acousticness",
                  "cast(energy as decimal(3,2)) as energy",
                  "key",
                  "cast(loudness as decimal(3,2)) as loudness",
                  "mode",
                  "cast(speechiness as decimal(5,2)) as speechiness",
                  "cast(acousticness as decimal(3,2)) as acousticness",
                  "cast(instrumentalness as decimal(3,2) ) as instrumentalness",
                  "cast (liveness as decimal(3,2) ) as liveness",
                  "cast(valence as decimal(3,2)) as valence",
                  "cast(tempo as decimal(3,2)) as tempo",
                  "type",
                  "id",
                  "uri",
                  "track_href",
                  "analysis_url",
                  "duration_ms",
                  "time_signature",
                  "genre",
                  "song_name",
                  "title",
                  "current_date() as dat_ref_carga")\
        .withColumn("rowid", monotonically_increasing_id())

### artists

In [1]:
artists = spark.read.option("header", "true").option('inferSchema', 'true').csv('/Users/eduardoalberto/LoadFile/archive/artists.csv')
artists = artists.select("id","followers",expr("regexp_replace(genres,'[^a-zA-Z0-9]',' ') as genres"),"name","popularity")
artists = artists.withColumn("rank",expr("rank() over (partition by name order by name)"))
artists = artists.withColumn("row",expr("row_number() over (partition by name order by name)"))
artists = artists.withColumn("dt_hoje",expr("cast(date_format(current_date(), 'yyyyMMdd') as int)"))
artists = artists.withColumn("dt_adesao",expr("cast(date_format(current_date()-3, 'yyyyMMdd')as int)"))

artistts = artists.filter("dt_adesao < dt_hoje")
artistts.toPandas()


NameError: name 'spark' is not defined